#Obtiene información de modelo TensorFlow.keras para poder utilizar en GoogleSheets

Inspirado de https://towardsdatascience.com/building-a-deep-neural-net-in-google-sheets-49cdaf466da0

Fuentes:

https://keras.io/2.15/api/models/model_saving_apis/weights_saving_and_loading/

https://alasko.medium.com/convert-simple-neuron-network-to-mathematician-notation-58a0d72f0337

https://docs.gspread.org/en/v6.0.1/user-guide.html

# Modelo.keras

In [1]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)



Mounted at /content/gdrive


In [2]:
#@title Cargar el Modelo.keras

import os.path
import shutil
from tensorflow import keras
from tensorflow.keras.models import load_model


path_modelo = '/content/gdrive/My Drive/IA/demoModelDeployment/modelo'  #@param {type:"string"}

#  carga el modelo a congelar
fn_path_modelo = path_modelo+"/model.keras"
model = load_model(fn_path_modelo)
print("\nModelo cargado de ", fn_path_modelo,"\n")

# cargar scaler (si existe)
fn_scaler = path_modelo+"/scaler.joblib"
if os.path.isfile(fn_scaler):
  print("\n* Se encuentra Scaler, pero no se puede emular!!! (por ahora) \n")

fn_clases = path_modelo+"/CLASES.txt"
CLASES = []
if os.path.isfile(fn_clases):
  with open(fn_clases, 'r') as f:
    # carga datos
    auxData = f.readlines()
  for c in auxData:
    CLASES.append( c.replace("\n", "") )
  print("\n* CLASES definidas cargado de ", fn_clases, ":")
  print("\t\t", CLASES, "\n")
else:
  print("\n* CLASES no encontradas en ", fn_clases, "\n")



Modelo cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/model.keras 


* CLASES definidas cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/CLASES.txt :
		 ['na', 'Setosa', 'Versicolor', 'Virginica'] 



In [3]:
#@title Extraer información sobre las capas del Modelo

capaEntradaDict = {}
capasOcultasList = []
capaSalidaDict = {}

def getLayerInfo(layer):
  lTipo = layer.__class__.__name__
  lConfig = layer.get_config()
  lWeights = list(layer.get_weights())
  return lTipo, lConfig, lWeights

def mostrarInfoCapaDict(capaConfig):
  for k in capaConfig:
    print("   ", k, ":", capaConfig[k])


# capa de entrada
layer = model.layers[0]
lTipo, lConfig, lWeights = getLayerInfo(model.layers[0])
print("\n-layer: ", lTipo)
assert ("InputLayer" in lTipo) # confirma que es capa de entrada
assert ("batch_shape" in lConfig) # y que tiene batchShape
capaEntradaDict["name"] = lConfig["name"]
capaEntradaDict["units"] = int(lConfig["batch_shape"][1])
mostrarInfoCapaDict(capaEntradaDict)

# capas ocultas y salida
for i in range(1, len(model.layers)):
  lTipo, lConfig, lWeights = getLayerInfo(model.layers[i])
  print("\n-layer: ", lTipo)
  if lTipo == "Dropout":
    print("\tse ignora porque no se usa para inferencia.")
    continue
  assert ("Dense" in lTipo) # por ahora sirve para capas lineales!
  capaDict = {}
  capaDict["name"] = lConfig["name"]
  if "activation" in lConfig:
    capaDict["activation"] = lConfig["activation"]
  if "units" in lConfig:
    capaDict["units"] = lConfig["units"]
  if len(lWeights)>0:
    capaDict["weights"] = lWeights[0]
  if lConfig["use_bias"]  and (len(lWeights)>1):
    capaDict["bias"] = lWeights[1]
  mostrarInfoCapaDict(capaDict)
  if i<(len(model.layers)-1):
    capasOcultasList.append(capaDict)
  else:
    capaSalidaDict = capaDict

print("")


-layer:  InputLayer
    name : input
    units : 4

-layer:  Dense
    name : hidd_1
    activation : linear
    units : 12
    weights : [[ 0.49487007 -0.03243791 -0.08926376 -0.08731144 -0.38394397 -0.41953573
   0.20948957  0.3171791  -0.7179932  -0.51479673  0.4876374  -0.3442734 ]
 [-0.56305385  0.56988186 -0.36473763  0.07394971 -0.6135267  -0.6083727
  -0.29908004  0.52444375 -0.20264171 -0.02547223  0.1386574  -0.5488239 ]
 [ 0.84721404 -0.5243547   0.19288243 -0.18629162  0.3733286   0.61428964
   0.58691895 -0.6640919   0.9650906   0.57780653  0.09069287  0.97251153]
 [-0.10653604 -0.10698458  0.79891026 -0.39699885  0.9542484   1.3191268
  -0.32494986 -0.878013    1.0533211   0.65719336 -1.4052417   1.204279  ]]
    bias : [-0.09899678  0.1981917  -0.7915295   0.4613356  -0.30440074 -0.7059444
 -0.40636718  0.5134458  -0.6529241  -0.5210035   0.3148326  -0.604451  ]

-layer:  Dense
    name : output
    activation : softmax
    units : 4
    weights : [[-0.35849687 -0.44425

# GoogleSheet

In [4]:
#@title Crear Planilla

import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default

# vuelve a auntentificar al usaurio
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


# crea nueva planilla y comporte con usuario actual
GoogleSpreadSheetname = 'RNA-IRIS'  #@param {type:"string"}
sh = gc.create(GoogleSpreadSheetname)
sh.share(creds.service_account_email, perm_type='user', role='writer')
# define locale US
# para poder manejar separador decimal "."
if sh.locale != "en_US":
  sh.update_locale("en_US")

print("Se creo la planilla ", GoogleSpreadSheetname, " y está disponible en https://docs.google.com/spreadsheets/u/0/")


Se creo la planilla  RNA-IRIS  y está disponible en https://docs.google.com/spreadsheets/u/0/


In [5]:
#@title Definir la Hoja de la Planilla a partir del Modelo

DEBUG = False #@param {type:"boolean"}

ws = sh.sheet1
try:
  ws.update_title("RNA")
except Exception as e:
  if DEBUG:
    print("\n--No se puede renombrar la hoja como RNA: ", e)


def getColLetter(col):
  return chr(64+col)

# datos de entrada
col = 1
col_letter = getColLetter(col+1)
prevCells = []
if DEBUG:
  print(capaEntradaDict)
ws.update_cell(1, col+1, 'Datos de Entrada')
for i in range(capaEntradaDict["units"]):
  ws.update_cell(i+2, col, capaEntradaDict["name"]+"_"+str(i+1))
  prevCells.append( col_letter + str(i+2) )
if DEBUG:
  print(prevCells)
# aplica formatos capa de entrada
ws.format("A1:"+col_letter+"1", {
    "backgroundColor": {
      "red": 0.20,
      "green": 0.70,
      "blue": 0.30
    },
    "horizontalAlignment": "CENTER",
    "textFormat": {
      "foregroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
      },
      "fontSize": 12,
      "bold": True
    }
})
ws.format("A2:A"+str(capaEntradaDict["units"]+1), {
    "horizontalAlignment": "CENTER",
    "textFormat": {
      "fontSize": 12,
      "bold": True
    }
})

# capas ocultas
col += 1
iniColCapaOculta = getColLetter(col+1)
for capaDict in capasOcultasList:
  col += 1
  col_letter = getColLetter(col)
  if DEBUG:
    print(capaDict)
  new_prevCells = []
  ws.update_cell(1, col, "Capa " + str(capaDict["name"]))
  for i in range(capaDict["units"]):
    strForm = "="
    for j in range(len(prevCells)):
      if len(strForm)>1:
        strForm = strForm + " + "
      strForm = strForm + "(" + prevCells[j] + " * " + str( capaDict["weights"][j][i] ) +")"
    if "bias" in capaDict:
      strForm = strForm + " + " + str( capaDict["bias"][i] )
    ws.update_cell(i+2, col, strForm)
    new_prevCells.append( col_letter + str(i+2) )
  prevCells = new_prevCells
  if DEBUG:
    print(prevCells)
  if "linear" not in capaDict["activation"]:
    # aplica la función que corresponde
    # (posibilidades 'exponential', 'relu', 'sigmoid', 'tanh' )
    col += 1
    col_letter = getColLetter(col)
    if DEBUG:
      print(col_letter)
    new_prevCells = []
    if "exponential" in capaDict["activation"]:
      funcGoogleSheet = "=EXP({0})"
    elif "relu" in capaDict["activation"]:
      funcGoogleSheet = "=SI({0}>=0,{0},0)"
    elif "sigmoid" in capaDict["activation"]:
      funcGoogleSheet = "=SIN({0})"
    elif "tanh" in capaDict["activation"]:
      funcGoogleSheet = "=TANH({0})"
    else:
      raise Exception("Falta implementar función ", capaSalidaDict["activation"])
    for j in range(len(prevCells)):
      ws.update_cell(j+2, col, funcGoogleSheet.format( prevCells[j] ))
      new_prevCells.append( col_letter + str(j+2) )
    prevCells = new_prevCells
    if DEBUG:
      print(prevCells)
# aplica formatos capas ocultas
ws.format(iniColCapaOculta+"1:"+col_letter+"1", {
    "backgroundColor": {
      "red": 0.15,
      "green": 0.15,
      "blue": 1.0
    },
    "horizontalAlignment": "CENTER",
    "textFormat": {
      "foregroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
      },
      "fontSize": 12,
      "bold": True
    }
})

# capa salida
col += 1
col_letter = getColLetter(col)
iniColCapaSalida = col_letter
if DEBUG:
  print(capaSalidaDict)
new_prevCells = []
ws.update_cell(1, col, "Capa " + str(capaSalidaDict["name"]))
for i in range(capaSalidaDict["units"]):
  strForm = "="
  for j in range(len(prevCells)):
    if len(strForm)>1:
      strForm = strForm + " + "
    strForm = strForm + "(" + prevCells[j] + " * " + str( capaSalidaDict["weights"][j][i] ) +")"
  if "bias" in capaSalidaDict:
    strForm = strForm + " + " + str( capaSalidaDict["bias"][i] )
  ws.update_cell(i+2, col, strForm)
  new_prevCells.append( col_letter + str(i+2) )
prevCells = new_prevCells
if DEBUG:
  print(prevCells)
if "softmax" in capaSalidaDict["activation"]:
  # calcula salida softmax
  ## calcula exponente relativo
  col += 1
  col_letter = getColLetter(col)
  if DEBUG:
    print(col_letter)
  new_prevCells = []
  for j in range(len(prevCells)):
    new_prevCells.append( "EXP(" + prevCells[j] + ")" )
  prevCells = new_prevCells
  strTodos = "+".join(prevCells)
  if DEBUG:
    print(strTodos)
  new_prevCells = []
  for j in range(len(prevCells)):
    ws.update_cell(j+2, col, "=" + prevCells[j]  + "/SUMA(" + strTodos + ")")
    new_prevCells.append( col_letter + str(j+2) )
  prevCells = new_prevCells
  if DEBUG:
    print(prevCells)
  ## determina mayor
  col += 1
  col_letter = getColLetter(col)
  if DEBUG:
    print(col_letter)
  strTodos = ",".join(prevCells)
  if DEBUG:
    print(strTodos)
  if len(CLASES)==0:
    # si no está definido CLASES lo crea
    for j in range(len(prevCells)):
      CLASES.append("clase "+str(j))
  for j in range(len(prevCells)):
    ws.update_cell(j+2, col, '=SI(' + prevCells[j] + '=max(' + strTodos + '),"' + CLASES[j] + ' [' + str(j) + ']","")')
  ws.format(col_letter+"2:"+col_letter+str(capaSalidaDict["units"]+1), {
    "horizontalAlignment": "CENTER",
    "textFormat": {
      "fontSize": 11,
      "bold": True
    }
  })
ws.format(iniColCapaSalida+"1:"+col_letter+"1", {
    "backgroundColor": {
      "red": 0.80,
      "green": 0.10,
      "blue": 0.80
    },
    "horizontalAlignment": "CENTER",
    "textFormat": {
      "foregroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
      },
      "fontSize": 12,
      "bold": True
    }
})

print("\nHoja lista para utilizar disponible en ", sh.url," \n")


Hoja lista para utilizar disponible en  https://docs.google.com/spreadsheets/d/1kXGKLNZzhon_wEsm_w8oKfofBffsQaKvERYyGgR1MfQ  

